In [1]:
import numpy as np
import pandas as pd
import scipy
from datetime import datetime
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set_palette('muted')

## Load Data

In [16]:
eth_raw = pd.read_csv('eth.csv')
eth_raw.tail()

,date,txVolume(USD),adjustedTxVolume(USD),txCount,marketcap(USD),price(USD),exchangeVolume(USD),generatedCoins,fees,activeAddresses,medianTxValue(USD),medianFee,averageDifficulty,paymentCount,blockSize,blockCount
2018-09-05,8.084180e+08,8.084180e+08,626957.0,2.910755e+10,286.05,2.390390e+09,20329.5000,362.380210,306686.0,0.429075,0.000175,3.435482e+15,296219.0,151363404,5956,NaN
2018-09-06,8.012842e+08,8.012842e+08,588959.0,2.357687e+10,231.65,2.097310e+09,20565.7500,511.158838,293094.0,1.853200,0.000251,3.399788e+15,300386.0,141998572,6048,NaN
2018-09-07,6.513843e+08,6.513843e+08,582184.0,2.336657e+10,229.53,1.678260e+09,20218.3125,506.599508,296253.0,0.154933,0.000261,3.399656e+15,260736.0,148491755,5913,NaN
2018-09-08,5.252607e+08,5.252607e+08,543337.0,2.218769e+10,217.91,1.517200e+09,20289.7500,404.495484,277064.0,0.000000,0.000260,3.375920e+15,225257.0,149634966,5936,NaN
2018-09-09,4.741100e+08,4.741100e+08,565631.0,2.020349e+10,198.38,1.585980e+09,20266.8750,433.078937,279792.0,0.000000,0.000269,3.209593e+15,232806.0,130475678,5899,NaN


In [2]:
eth_price_raw = pd.read_csv('ethereum_price.csv')
print(eth_price_raw.info())
eth_price_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 7 columns):
Date          929 non-null object
Open          929 non-null float64
High          929 non-null float64
Low           929 non-null float64
Close         929 non-null float64
Volume        929 non-null object
Market Cap    929 non-null object
dtypes: float64(4), object(3)
memory usage: 50.9+ KB
None


,Date,Open,High,Low,Close,Volume,Market Cap
0,"Feb 20, 2018",943.57,965.26,892.95,895.37,"2,545,260,000","92,206,500,000"
1,"Feb 19, 2018",921.67,957.78,921.55,943.87,"2,169,020,000","90,047,700,000"
2,"Feb 18, 2018",973.35,982.93,915.45,923.92,"2,567,290,000","95,077,100,000"
3,"Feb 17, 2018",944.75,976.60,940.75,974.12,"2,525,720,000","92,264,000,000"
4,"Feb 16, 2018",934.79,950.00,917.85,944.21,"2,369,450,000","91,272,100,000"


In [8]:
eth = eth_price_raw.copy()
eth['Date'] = pd.to_datetime(eth['Date'], format='%b %d, %Y')
eth.sort_values('Date', inplace=True)
eth.reset_index(drop=True, inplace=True)
print(eth.info())
eth.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 7 columns):
Date          929 non-null datetime64[ns]
Open          929 non-null float64
High          929 non-null float64
Low           929 non-null float64
Close         929 non-null float64
Volume        929 non-null object
Market Cap    929 non-null object
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 50.9+ KB
None


,Date,Open,High,Low,Close,Volume,Market Cap
0,2015-08-07,2.830000,3.540000,2.520000,2.770000,"164,329",-
1,2015-08-08,2.790000,2.800000,0.714725,0.753325,"674,188","167,911,000"
2,2015-08-09,0.706136,0.879810,0.629191,0.701897,"532,170","42,637,600"
3,2015-08-10,0.713989,0.729854,0.636546,0.708448,"405,283","43,130,000"
4,2015-08-11,0.708087,1.130000,0.663235,1.070000,"1,463,100","42,796,500"


In [9]:
eth['Volume'] = pd.to_numeric(eth['Volume'].str.replace(',', ''), errors='coerce')
eth['Market Cap'] = pd.to_numeric(eth['Market Cap'].str.replace(',', ''), errors='coerce')
eth.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,2015-08-07,2.830000,3.540000,2.520000,2.770000,164329,NaN
1,2015-08-08,2.790000,2.800000,0.714725,0.753325,674188,167911000.0
2,2015-08-09,0.706136,0.879810,0.629191,0.701897,532170,42637600.0
3,2015-08-10,0.713989,0.729854,0.636546,0.708448,405283,43130000.0
4,2015-08-11,0.708087,1.130000,0.663235,1.070000,1463100,42796500.0


## Exogenous Variables
- Etherem related
    - Volume
    - Market Cap
- Other Cryptocurrencies
    - BTC: Bitcoin
    - XRP: Ripple
    - EOS: EOS.IO
    - LTC: Litecoin
    - XLM: Stellar
    - XMR: Monero
- Other Indices
    - VIX
    - DXY
    - EFFR

In [10]:
df = eth_price_raw[['Date', 'Close', 'Volume', 'Market Cap']].copy()
df.columns = ['date', 'eth', 'eth_volume', 'eth_market_cap']
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y')
df.sort_values('date', inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,date,eth,eth_volume,eth_market_cap
0,2015-08-07,2.770000,"164,329",-
1,2015-08-08,0.753325,"674,188","167,911,000"
2,2015-08-09,0.701897,"532,170","42,637,600"
3,2015-08-10,0.708448,"405,283","43,130,000"
4,2015-08-11,1.070000,"1,463,100","42,796,500"


### **[VIX: CBOE Volatility Index](https://en.wikipedia.org/wiki/VIX)**
- Measure of stock market's expectation of volatility implied by S&P 500 index options, aka 'fear index'
- [Data Source](http://www.cboe.com/products/vix-index-volatility/vix-options-and-futures/vix-index/vix-historical-data)

In [ ]:
vix_raw = pd.read_csv('vixcurrent.csv', header=1)
print(vix_raw.info())

In [ ]:
vix_raw[:1]

In [ ]:
vix_close = vix_raw[['Date', 'VIX Close']].copy()
vix_close['Date'] = pd.to_datetime(vix_close['Date'], format='%m/%d/%Y')
vix_close = vix_close.loc[(vix_close['Date'] >= '2015-08-07')
                          & (vix_close['Date'] <= '2018-02-20')]
vix_close.sort_values('Date', inplace=True)
vix_close.reset_index(drop=True, inplace=True)
vix_close.columns = ['date', 'vix']
vix_close.head()

In [ ]:
df_ex = df.merge(vix_close, how='outer', on='date')
df_ex.vix.ffill(inplace=True)
#df_04ex.vix[0]= vix_raw.loc[vix_raw['Date'] == '11/24/2017', ['VIX Close']].values
print(df_ex.vix.isnull().sum())
df_ex.head()

### **[DXY: US Dollar Index](https://en.wikipedia.org/wiki/U.S._Dollar_Index)**
- Measure of value of USD relative to a basket of foreign currencies
- [Data Source](https://quotes.wsj.com/index/DXY/historical-prices)

In [ ]:
dxy_raw = pd.read_csv('dxy_historical.csv')
dxy_raw[:1]

In [ ]:
dxy_raw.info()

In [ ]:
dxy = dxy_raw[['Date', ' Close']].copy()
dxy['Date'] = pd.to_datetime(dxy['Date'], format='%x')
dxy = dxy.loc[(dxy['Date'] >= '2015-08-07')
              & (dxy['Date'] <= '2018-02-20')]
dxy.sort_values('Date', inplace=True)
dxy.reset_index(drop=True, inplace=True)
dxy.columns = ['date', 'dxy']
dxy.head()

In [ ]:
df_ex = df_ex.merge(dxy, how='outer', on='date')
df_ex.ffill(inplace=True)

# get first value
#dxy_raw['Date'] = pd.to_datetime(dxy_raw['Date'], format='%x')
#df_04ex.dxy[0] = dxy_raw.loc[dxy_raw['Date'] == '11/24/2017', [' Close']].values
print(df_ex.dxy.isnull().sum())
df_ex.head()

### **[EFFR: Federal Funds Rate](https://en.wikipedia.org/wiki/Federal_funds_rate)**
- The interest rate at which depository institutions (banks and credit unions) lend reserve balances to other depository institutions overnight, on an uncollateralized basis 
- [Data Source](https://fred.stlouisfed.org/series/EFFR)

In [ ]:
effr_raw = pd.read_csv('EFFR.csv')
effr_raw.head()

In [ ]:
effr = effr_raw.copy()
effr.columns = ['date', 'effr']
effr['date'] = pd.to_datetime(effr['date'], format='%Y-%m-%d')
effr.sort_values('date', inplace=True)
effr = effr.loc[(effr['date'] >= '2015-08-07') & (effr['date'] <= '2018-02-20')]
effr.reset_index(drop=True, inplace=True)
effr.head()

In [ ]:
df_ex = df_ex.merge(effr, how='outer', on='date')
df_ex.ffill(inplace=True)
print(df_ex.effr.isnull().sum())
df_ex.head()